In [112]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px

# Лабораторная работа №3
## Задание 5
## Кондрашин Тимофей 211-365 Вариант 11 (27.04.2023)

### Данные

По данным выборки проверить с помощью критерия Пирсона при уровне значимости α гипотезу:
- о показательном;
- равномерном;
- нормальном законе распределения генеральной совокупности.

В ответе привести:
1) выбранную гипотезу о виде закона распределения;
2) вычисленное значение критерия;
3) критическое значение;
4) вывод о принятии или не принятии гипотезы.


In [113]:
data = pd.DataFrame([15.6, 29.1, -5.3, 8.8, 8.7, 2.5, 10.3, 14.4, 21.5, 9.9, 33.5, 15.9, 8.0, 21.4, 9.9, 1.7, 11.2, 9.0, 4.5, 26.2, 3.2, 11.0, 18.3, 5.7, 21.4, 14.4, 17.5, -1.4, 0.3, 17.5, 8.8, 16.0, 0.7, 4.2, 12.2, 14.6, 16.8, 15.2, 17.0, 17.2, -1.7, 24.0, 23.6, 33.2, 21.4, 10.4, 1.5, 8.1, 12.2, 8.2, 17.6, 9.5, 22.9, 9.0, 30.0, 18.2, -10.0, -4.6, 21.4, 20.1, 18.4, 24.3, 7.5, 12.5, 7.7, 0.2, -11.0, 24.2, 24.8, 6.0, 6.6, 15.9, 15.5, 18.6, 4.3, 21.4, 8.0, 13.9, 3.9, 25.2, 16.8, 14.2, 11.8, 16.4, 14.2, 2.9, -2.3, 14.8, 19.7, -1.3, 6.0, 14.7, 9.5, 2.8, 25.0, -0.4, 9.9, 16.3, 4.8, 16.0])
n = len(data)
a = 0.025

data.T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,15.6,29.1,-5.3,8.8,8.7,2.5,10.3,14.4,21.5,9.9,...,6.0,14.7,9.5,2.8,25.0,-0.4,9.9,16.3,4.8,16.0


In [114]:
print(f'n = {n}, a = {a}')

n = 100, a = 0.025


## Шаг 1: Формулировка гипотезы

$H_0$: закон распределения генеральной совокупности является нормальным.
$H_1$: закон распределения генеральной совокупности является показательным.

## Шаг 2: Определение уровня значимости и степеней свободы

#### Уровень значимости:
a = 0.025

In [115]:
a

0.025

## Шаг 2: Вычисление характеристик выборки.

### Вычисление среднего значения

$$\overline{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

In [116]:
mean = data.mean()
mean

0    12.221
dtype: float64

### Вычисление дисперсии

$$s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \overline{x})^2$$

In [117]:
variance = data.var()
variance

0    82.672787
dtype: float64

### Вычисление стандартного отклонения

$$s = \sqrt{s^2}$$

In [118]:
std = data.std()
std

0    9.092458
dtype: float64

## Шаг 3: Проверка гипотезы

Для проверки гипотезы о нормальности распределения воспользуемся критерием согласия хи-квадрат. Сначала необходимо разбить интервальный ряд на интервалы.

Для этого найдём количество интервалов:

$$k = \lceil \log_2 n + 1 \rceil$$

In [119]:
k = int(np.ceil(np.log2(n) + 1))
k

8

In [ ]:
fig = px.histogram(data, labels={'count': 'Частота', 'value': 'Значение'}, nbins=k, title='Интервальный ряд', marginal="violin", text_auto=True)
fig.show()

##### Судя по графику, можно сделать вывод, что распределение является нормальным.

Теперь можно разбить интервальный ряд на интервалы:

$$\Delta = \frac{max(x) - min(x)}{k}$$

In [ ]:
delta = (data.max()[0] - data.min()[0]) / k
delta

Теперь можно построить интервальный ряд:

In [ ]:
intervals = []
for i in range(k):
    intervals.append([i-1 * delta,(i) * delta])
intervals

Далее, необходимо вычислить выборочные частоты попадания значений в каждый интервал:

$$p_i = \frac{n_i}{n}$$

In [ ]:
p = []
for i in range(k):
    p.append(len(data[(data[0] >= i-1 * delta) & (data[0] < (i) * delta)]) / n)
p

[0.07, 0.19, 0.42, 0.64, 0.81, 0.88, 0.88, 0.88]

Затем нужно вычислить теоретические (ожидаемые) частоты попадания значений в каждый интервал, если бы распределение было нормальным. Для этого воспользуемся формулой:
$$P_i = \Phi \left( \frac{x_{i+1} - \overline{x}}{s} \right) - \Phi \left( \frac{x_{i} - \overline{x}}{s} \right)$$

In [ ]:
P = []
for i in range(k):
    P.append(sp.stats.norm.cdf((intervals[i][1] - mean) / std) - sp.stats.norm.cdf((intervals[i][0] - mean) / std))
P

[array([0.06421979]),
 array([0.19953411]),
 array([0.4107362]),
 array([0.63638397]),
 array([0.80021794]),
 array([0.87693641]),
 array([0.89251131]),
 array([0.88053548])]

In [ ]:
intervals_t = pd.DataFrame(intervals, columns=['min', 'max'])
p_t = pd.DataFrame(p, columns=['p (выборочные частоты)'])
P_t = pd.DataFrame(P, columns=['P (ожидаемые частоты)'])
intervals_t.join(p_t).join(P_t).T

,0,1,2,3,4,5,6,7
min,-5.56250,-4.562500,-3.562500,-2.562500,-1.562500,-0.562500,0.437500,1.437500
max,0.00000,5.562500,11.125000,16.687500,22.250000,27.812500,33.375000,38.937500
p (выборочные частоты),0.07000,0.190000,0.420000,0.640000,0.810000,0.880000,0.880000,0.880000
P (ожидаемые частоты),0.06422,0.199534,0.410736,0.636384,0.800218,0.876936,0.892511,0.880535


## Шаг 4: Вычисление значения статистики критерия

Вычислим значение статистики критерия $\chi^2$:

$$\chi^2 = \sum_{i=1}^{k} \frac{(p_i - P_i)^2}{P_i}$$

In [ ]:
chi2 = (((np.array(p) - np.array(P)) ** 2).sum()) / np.array(P).sum()
chi2

2.5668656166346002

## Шаг 5: Определение критического значения и принятие решения

#### Уровень значимости:
a = 0.025

In [ ]:
a

0.025

#### Степени свободы:
$$df = k - 1$$
где
- $k$ - количество интервалов,
- $p$ - количество параметров распределения, которые были оценены по выборке.

In [128]:
df = k - 1
df

7

Таблица распределения хи-квадрат ($\chi^2$) со степенями свободы 7 и уровнем значимости 0.025 даёт критическое значение $\chi^2 = 16.013$.

Так как вычисленное значение статистики критерия Пирсона ($\chi^2 = 2.5668656166346002$) не превышает критическое значение ($\chi^2 = 16.013$), то мы принимаем нулевую гипотезу $H_0$ и отвергаем альтернативную гипотезу $H_1$. Это означает, что закон распределения генеральной совокупности является нормальным.


## Шаг 5: Вывод

Таким образом, на основе данных выборки мы можем с уверенностью утверждать, что закон распределения генеральной совокупности является нормальным.